# 시계열 데이터 예측 알고리즘 성능 비교

### 비콘 메시지의 수신 시간 간격 계산 함수

In [1]:
def calc_INT(time):
    INT = np.array([])
    
    for i in range(1, len(time)):
        INT = np.append(INT, df_time[i] - df_time[i-1])
    
    return INT

In [2]:
def percentile(l, s):
    if l < s:
        return ( l / s ) / 980
    else:
        return ( s / l ) / 980

In [3]:
import pandas as pd
import numpy as np
import time
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
import warnings
warnings.filterwarnings(action='ignore')

df = pd.read_csv('./data/gal_data1.csv')
df_time = df['time']

INT = calc_INT(df_time)
INT_p = INT[21:]

### 지수평활, 홀트-선형 추세, 홀트-지수 추세, 홀트-윈터스 계절 기법 성능 비교

In [4]:
SES = np.array([])
HM = np.array([]) 
HME = np.array([])
HW = np.array([])

SES_time = 0
HM_time = 0
HME_time = 0
HW_time = 0

In [5]:
start = time.time()
for i in range(20, len(INT)):
    fit1 = SimpleExpSmoothing(INT[:i], initialization_method="estimated").fit()
    SES = np.append(SES, fit1.forecast(1)[0])
SES_time = time.time() - start

start = time.time()
for i in range(20, len(INT)):
    fit2 = Holt(INT[:i], initialization_method="estimated").fit()
    HM = np.append(HM, fit2.forecast(1)[0])
HM_time = time.time() - start

start = time.time()
for i in range(20, len(INT)):
    fit3 = Holt(INT[:i], exponential=True, initialization_method="estimated").fit()
    HME = np.append(HME, fit3.forecast(1)[0])
HME_time = time.time() - start

start = time.time()
for i in range(20, len(INT)):
    fit4 = ExponentialSmoothing(INT[:i], seasonal_periods=4, trend="add", seasonal="add", use_boxcox=True, initialization_method="estimated").fit()
    HW = np.append(HW, fit4.forecast(1)[0])
HW_time = time.time() - start


In [6]:
SES_p = np.array([])
HM_p = np.array([]) 
HME_p = np.array([])
HW_p = np.array([])

for i in range(0, len(INT_p)):
    SES_p = np.append(SES_p, percentile(INT_p[i], SES[i]))
    HM_p = np.append(HM_p, percentile(INT_p[i], HM[i]))    
    HME_p = np.append(HME_p, percentile(INT_p[i], HME[i]))    
    HW_p = np.append(HW_p, percentile(INT_p[i], HW[i]))    

In [7]:
print('SES Accuracy: ', np.sum(SES_p))
print('SES Time: ', SES_time)
print('')
print('Holt Linear Accuracy: ', np.sum(HM_p))
print('HM Time: ', HM_time)
print('')
print('Holt Exponential Accuracy: ', np.sum(HME_p))
print('HME Time: ', HME_time)
print('')
print('Holt-Winters Accuracy: ', np.sum(HW_p))
print('HW Time: ', HW_time)

SES Accuracy:  0.8311869417821911
SES Time:  6.08599853515625

Holt Linear Accuracy:  0.8415306038975895
HM Time:  35.355321407318115

Holt Exponential Accuracy:  0.8409678176123869
HME Time:  40.081098318099976

Holt-Winters Accuracy:  0.9115327366172779
HW Time:  171.71415400505066
